In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from root_numpy import root2array, rec2array

branch_names = '''Xmomentum,Ymomentum,Momentum,Energy,MvdDEDX,MvdHits,SttMeanDEDX,SttHits,GemHits,
TofStopTime,TofM2,TofTrackLength,TofQuality,DrcThetaC,DrcQuality,
DiscThetaC,DiscQuality,EmcRawEnergy,EmcCalEnergy,EmcQuality,EmcNumberOfCrystals,
EmcNumberOfBumps,EmcModule,EmcZ20,EmcZ53,EmcLat,EmcE1,EmcE9,EmcE25,MuoQuality,MuoIron,
MuoMomentumIn,MuoNumberOfLayers,MuoModule,MuoHits,DegreesOfFreedom,ChiSquared,Theta'''.split(",")

branch_names_2 = '''momentumx,momentumy,momentum,energy,MvdDEDX,MvdHits,SttMeanDEDX,SttHits,GemHits,\
TofStopTime,TofM2,TofTrackLength,TofQuality,DrcThetaC,DrcQuality,\
DiscThetaC,DiscQuality,EmcRawEnergy,EmcCalEnergy,EmcQuality,EmcNumberOfCrystals,\
EmcNumberOfBumps,EmcModule,EmcZ20,EmcZ53,EmcLat,EmcE1,EmcE9,EmcE25,MuoQuality,MuoIron,\
MuoMomentumIn,MuoNumberOfLayers,MuoModule,MuoHits,DegreesOfFreedom,ChiSquared'''.split(",")

branch_names = [c.strip() for c in branch_names]
branch_names = list(branch_names)

def loadtrainingdata(name):
    filestring="/home/lewis/particles/box_500k_" + name + ".root"
    vals_1 = root2array(filestring, 't1', branch_names)
    vals_1 = rec2array(vals_1)
    filestring="/home/lewis/particles/box_500k_anti_" + name + ".root"
    vals_2 = root2array(filestring, 't1', branch_names)
    vals_2 = rec2array(vals_2)
    filestring="/home/lewis/particles/evt_500k_" + name + ".root"
    vals_3 = root2array(filestring, 't1', branch_names)
    vals_3 = rec2array(vals_3)
    filestring="/home/lewis/particles/dpmbkg_1M_" + name + ".root"
    vals_4 = root2array(filestring, 't1', branch_names)
    vals_4 = rec2array(vals_4)[:17644,:]
    vals = np.concatenate((vals_1,vals_2,vals_3,vals_4))
    print(vals.shape)
    return vals

def loadtestingdata(name):
    filestring="/home/lewis/150k_particles/150k/particles/" + name + "_150k_tree.root"
    vals_4 = root2array(filestring, 't1', branch_names_2)
    vals_4 = rec2array(vals_4)
    filestring="/home/lewis/150k_particles/150k/antiparticles/P" + name + "_theta_150k_tree.root"
    vals_5 = root2array(filestring, 't1', branch_names_2)
    vals_5 = rec2array(vals_5)
    vals = np.concatenate((vals_4,vals_5))
    print (vals.shape)
    return vals

#for particle in particle_list:
electrons = loadtrainingdata('electrons')
pions = loadtrainingdata('pions')
muons = loadtrainingdata('muons')
kaons = loadtrainingdata('kaons')
protons = loadtrainingdata('protons')

In [ ]:
def createdataframe(electrons,pions,muons,kaons,protons,branch_names):
    ###################################################################################
    X = np.concatenate((electrons, pions, muons, kaons, protons))
    y = np.concatenate(( np.zeros(electrons.shape[0]),np.ones(pions.shape[0]), (2*np.ones(muons.shape[0])), (3*np.ones(kaons.shape[0])), (4*np.ones(protons.shape[0])) ))
    df = pd.DataFrame(np.hstack((X, y.reshape(y.shape[0], -1))),columns=branch_names+['temp'])
    
    
    # added features
    df['Pt'] = np.sqrt( df.loc[:,'Xmomentum']**2  + df.loc[:,'Ymomentum']**2 )
    df['EoverP'] = df.loc[:,'EmcCalEnergy']/df.loc[:,'Momentum']

    df = df.drop(['Xmomentum'], axis=1)
    df = df.drop(['Ymomentum'], axis=1)

    df['labels'] = df.loc[:,'temp']
    df = df.drop(['temp'], axis=1)
    branch_names = list(df.columns.values[0:-1])
    pd.set_option('display.max_columns', None)
    return df

df = createdataframe(electrons,pions,muons,kaons,protons,branch_names)
mask = df['Momentum'] < 15
df = df[mask]
df.tail(10)

In [ ]:
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier

X_train,X_test, y_train,y_test = train_test_split(df.iloc[:,0:-1], df.iloc[:,-1], test_size=0.25, random_state=0)

xgb = XGBClassifier(max_depth=12, learning_rate=0.15, n_jobs=4, tree_method='exact', verbose=True)

In [ ]:
%time clf = xgb.fit(X_train, y_train,verbose=True)
print('accuracy on training set: %.2f' % (clf.score(X_train, y_train)*100) + '%')
print('accuracy on testing set : %.2f' % (clf.score(X_test, y_test)*100) + '%')

In [ ]:
from sklearn.externals import joblib

import pickle
joblib.dump(clf, '/home/lewis/particles/xgb_1.pkl')
print("the xgb model has been saved succesfully")

In [ ]:
from __future__ import print_function
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
#from sklearn.decomposition import PCA

import keras
from keras import optimizers
#from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Reshape, BatchNormalization
from keras.optimizers import RMSprop, Adam, Adagrad
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.utils.np_utils import to_categorical

X_train,X_test, y_train,y_test = train_test_split(df.iloc[:,0:-1], df.iloc[:,-1], test_size=0.20, random_state=42)

# scale the features
scaler = StandardScaler()
#X_train = scaler.fit_transform(X_train)
#X_test = scaler.transform(X_test)

NComponents = X_train.shape[1]
'''
# use pca
pca = PCA().fit(X_train)
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.axvline(x=22, color='r')
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance');

NComponents = 22
pca = PCA(n_components=NComponents)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)
'''

# some variables
batch_size = 4096
num_classes = 5
epochs = 5000

# one hot encoding
#y_train = keras.utils.to_categorical(y_train, num_classes)
#y_test = keras.utils.to_categorical(y_test, num_classes)


# define model
model = Sequential()
model.add(BatchNormalization(input_shape=(NComponents,)))
model.add(Dense(512, activation='tanh'))
model.add(Dense(256, activation='tanh'))
model.add(Dense(128, activation='tanh'))
model.add(Dense(64, activation='tanh'))
model.add(Dense(32, activation='tanh'))
model.add(Dropout(0.25))
model.add(Dense(16, activation='tanh'))
model.add(Dropout(0.25))
model.add(Dense(num_classes, activation='softmax'))

model.summary()
model.compile(loss='sparse_categorical_crossentropy',optimizer=keras.optimizers.Adam(),metrics=['accuracy'])

print(X_train.shape)
print(y_train.shape)

In [ ]:
import time

t0 = time.time()

history = model.fit(X_train , y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    callbacks=[EarlyStopping(verbose=True, patience=500, monitor='val_loss')],
                    validation_data=(X_test , y_test))

t = time.time() - t0

print("training time = ", t, " s")
print("********************************")

In [ ]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:%.2f' %score[0])
print('Test accuracy:%.2f' % score[1])

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

ann = history.history
plt.grid()
plt.plot(100 * np.array(ann['acc']), label='training')
plt.plot(100 * np.array(ann['val_acc']), label='validation')
plt.xlim(0)
plt.xlabel('Iterations', fontsize=20)
plt.ylabel('Accuracy %', fontsize=20)
plt.legend(loc='lower right', fontsize=20)
plt.savefig('history.png')

In [ ]:
print ('Testing...')
yhat = model.predict(X_test, verbose = True, batch_size = 4096)
#yhat = model.predict_proba(X_test)
print(yhat)

In [ ]:
# CONFUSION MATRIX 5 classes (5 different particles)
yhat_cls = np.argmax(yhat, axis=1)
from sklearn.metrics import confusion_matrix
import itertools

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues,file='0'):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()
    plt.savefig('confusion_matrix_'+file+'.png')
   

In [ ]:
 
# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test, yhat_cls)
np.set_printoptions(precision=2)
plot_confusion_matrix(cnf_matrix, classes=['e','pi','mu','ka','p'],
                      normalize=True,
                      title='Normalized Confusion Matrix',file='train_xgb')

In [ ]:

#for particle in particle_list:
electrons = loadtestingdata('electrons')
pions = loadtestingdata('pions')
muons = loadtestingdata('muons')
kaons = loadtestingdata('kaons')
protons = loadtestingdata('protons')

df = createdataframe(electrons,pions,muons,kaons,protons,branch_names)

In [ ]:
#df = df.drop(['Theta'], axis=1)
X_test, y_test = df.iloc[:,0:-1], df.iloc[:,-1]

In [ ]:
X_test = X_test.drop(['Theta'],axis=1)
Theta = df.iloc[:,-4]
X_test.head()

In [ ]:
#X_test = scaler.transform(X_test)
#X_test = pca.transform(X_test)
print ('Testing...')
yhat = model.predict(X_test.drop(['Theta'],axis=1), verbose = True, batch_size = 4096)
print(yhat)

In [ ]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:%.2f' %score[0])
print('Test accuracy:%.2f' % score[1])

# CONFUSION MATRIX 5 classes (5 different particles)
yhat_cls = np.argmax(yhat,axis=1)
    
# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test, yhat_cls)
np.set_printoptions(precision=2)
plot_confusion_matrix(cnf_matrix, classes=['e','pi','mu','ka','p'],
                      normalize=True,
                      title='Normalized Confusion Matrix',file='test_xgb')

In [ ]:
model.save('/home/lewis/particles/model_7.hdf5')

In [ ]:
# here we will find the accuracy of the model for certain momentum ranges
#from sklearn.metrics import accuracy_score

pmax = 5.0
step_size = 0.05
mom_range = np.arange(0.0,pmax+step_size,step_size)

thetamax = 180.0
theta_step = 2.5
theta_range = np.arange(0.0,thetamax+theta_step,theta_step)

def accuracy_vs_momentum(pdf,pX_test,py_test):
    scores = []
    #pX_test = pX_test.drop(['Theta'], axis=1)
    for momentum in mom_range:
        pmask = (pdf['Momentum'] <= (momentum+step_size)) & (pdf['Momentum'] > momentum)
        score = model.evaluate(pX_test[pmask], py_test[pmask], verbose=0)
        if not score:
            scores.append(0) 
        else:
            scores.append(score[1])
    return scores[1:]


def accuracy_vs_theta(pdf,pX_test,py_test,Theta):
    scores = []
    for theta in theta_range:
        mask = (Theta <= (theta+theta_step)) & (Theta > theta)
        score = model.evaluate(pX_test[mask], py_test[mask], verbose=0)
        if not score:
            scores.append(0) 
        else:
            scores.append(score[1])
    return scores[1:]

In [ ]:
# find the accuracies for the different particles

mask = df['labels'] == 0.0
e_score = accuracy_vs_momentum(df[mask],X_test[mask],y_test[mask])
mask = df['labels'] == 2.0
mu_score = accuracy_vs_momentum(df[mask],X_test[mask],y_test[mask])
mask = df['labels'] == 1.0
pi_score = accuracy_vs_momentum(df[mask],X_test[mask],y_test[mask])
mask = df['labels'] == 3.0
ka_score = accuracy_vs_momentum(df[mask],X_test[mask],y_test[mask])
mask = df['labels'] == 4.0
p_score = accuracy_vs_momentum(df[mask],X_test[mask],y_test[mask])
mom_range = mom_range[1:]

In [ ]:
# Create a scatter plot of the model's accuracy versus momentum
plt.scatter(mom_range, e_score,s=12,color='aqua', label='$e^{-}$')
plt.scatter(mom_range, pi_score,s=12,color='darkorange', label='$\pi^{-}$')
plt.scatter(mom_range, mu_score,s=12,color='cornflowerblue', label='$\mu^{-}$')
plt.scatter(mom_range, ka_score,s=12,color='black', label='$k^{-}$')
plt.scatter(mom_range, p_score,s=12,color='green', label='$\overline{p}$')
plt.title('Neural Network Accuracy vs Momentum',fontsize=16)
plt.grid()
plt.ylim(0,1)
plt.xlim(0,pmax)
plt.xlabel('Momentum in GeV/C',fontsize=16)
plt.ylabel('Accuracy',fontsize=16)
plt.legend(loc='lower right', fontsize=14)
plt.tight_layout()
plt.savefig('accuracy_v_momentum_full.png')
plt.show()

In [ ]:
# find the accuracies for the different particles

mask = df['labels'] == 0.0
e_score = accuracy_vs_theta(df[mask],X_test[mask],y_test[mask],Theta[mask])
mask = df['labels'] == 2.0
mu_score = accuracy_vs_theta(df[mask],X_test[mask],y_test[mask],Theta[mask])
mask = df['labels'] == 1.0
pi_score = accuracy_vs_theta(df[mask],X_test[mask],y_test[mask],Theta[mask])
mask = df['labels'] == 3.0
ka_score = accuracy_vs_theta(df[mask],X_test[mask],y_test[mask],Theta[mask])
mask = df['labels'] == 4.0
p_score = accuracy_vs_theta(df[mask],X_test[mask],y_test[mask],Theta[mask])
theta_range = theta_range[1:]

In [ ]:
# Create a scatter plot of the model's accuracy versus theta
plt.scatter(theta_range, e_score,s=12,color='aqua', label='$e^{-}$')
plt.scatter(theta_range, pi_score,s=12,color='darkorange', label='$\pi^{-}$')
plt.scatter(theta_range, mu_score,s=12,color='cornflowerblue', label='$\mu^{-}$')
plt.scatter(theta_range, ka_score,s=12,color='black', label='$k^{-}$')
plt.scatter(theta_range, p_score,s=12,color='green', label='$\overline{p}$')
plt.title('Neural Network Accuracy vs Theta',fontsize=16)
plt.grid()
plt.ylim(0,1)
plt.xlim(0,thetamax)
plt.xlabel('Theta in Degrees',fontsize=16)
plt.ylabel('Accuracy',fontsize=16)
plt.legend(loc='lower center', fontsize=14)
plt.tight_layout()
plt.savefig('accuracy_v_theta_full.png')
plt.show()

In [ ]:
from xgboost import XGBClassifier
from sklearn.externals import joblib
clf = joblib.load('/home/lewis/particles/xgb_0.pkl')

import keras
model = keras.models.load_model('/home/lewis/particles/model_5.hdf5')

In [ ]:
# Compute ROC curve and ROC area for each class
from sklearn.metrics import roc_curve, auc
y_score = clf.predict_proba(X_test)
from sklearn.preprocessing import label_binarize
y_test = label_binarize(y_test, classes=[0, 1, 2, 3, 4])
n_classes = y_test.shape[1]
import matplotlib.pyplot as plt
%matplotlib tk

lw=2
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

from itertools import cycle

plt.plot(fpr[0], tpr[0], color='aqua', lw=lw, label='ROC for {0} (area = {1:0.2f})'.format('$e^{-}$',roc_auc[0]))
plt.plot(fpr[1], tpr[1], color='darkorange', lw=lw, label='ROC for {0} (area = {1:0.2f})'.format('$\pi^{-}$',roc_auc[1]))
plt.plot(fpr[2], tpr[2], color='cornflowerblue', lw=lw, label='ROC for {0} (area = {1:0.2f})'.format('$\mu^{-}$',roc_auc[2]))
plt.plot(fpr[3], tpr[3], color='black', lw=lw, label='ROC for {0} (area = {1:0.2f})'.format('$k^{-}$',roc_auc[3]))
plt.plot(fpr[4], tpr[4], color='green', lw=lw, label='ROC for {0}   (area = {1:0.2f})'.format('$\overline{p}$',roc_auc[4]))

plt.xlim([0.0, 0.5])
plt.ylim([0.5, 1.0])
plt.xlabel('False Positive Rate', fontsize=22)
plt.ylabel('True Positive Rate', fontsize=22)
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc='lower right', fontsize=14)
plt.savefig('full_roc_curve.png')

In [ ]:
# Compute precision recall curve for each class
from sklearn.metrics import precision_recall_curve

fpr = dict()
tpr = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = precision_recall_curve(y_test[:, i], y_score[:, i])

plt.plot(fpr[0], tpr[0], color='aqua', lw=lw, label='Electrons')
plt.plot(fpr[1], tpr[1], color='darkorange', lw=lw, label='Pions')
plt.plot(fpr[2], tpr[2], color='cornflowerblue', lw=lw, label='Muons')
plt.plot(fpr[3], tpr[3], color='black', lw=lw, label='Kaons')
plt.plot(fpr[4], tpr[4], color='green', lw=lw, label='Protons')

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('Recall', fontsize=22)
plt.ylabel('Precision', fontsize=22)
plt.legend(loc='lower right', fontsize=14)
plt.savefig('precision_recall_curve.png')